In [12]:
import pprint

### Load components

In [15]:
import requests
from pydantic import BaseModel, Field
import datetime

from langchain.agents import tool

# Define the input schema
class GreetInput(BaseModel):
    name: str = Field(
        ..., 
        title="Name of the user", 
        description="Name of the user to get the current info for.")

@tool(args_schema=GreetInput)
def get_current_user(name: str):
    """Fetch and greet to the current user."""
    return f"Hi {name}, it is the pleasure to meet you!"

In [18]:
print(get_current_user.name)
print(get_current_user.description)
pprint.pprint(get_current_user.args)
print(get_current_user("Weber"))

get_current_user
Fetch and greet to the current user.
{'name': {'description': 'Name of the user to get the current info for.',
          'title': 'Name of the user',
          'type': 'string'}}
Hi Weber, it is the pleasure to meet you!


In [19]:
from langchain_core.utils.function_calling import convert_to_openai_function

pprint.pprint(convert_to_openai_function(get_current_user))

{'description': 'Fetch and greet to the current user.',
 'name': 'get_current_user',
 'parameters': {'properties': {'name': {'description': 'Name of the user to '
                                                       'get the current info '
                                                       'for.',
                                        'type': 'string'}},
                'required': ['name'],
                'type': 'object'}}


In [21]:
import wikipedia


@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
        summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")

    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [22]:
print(search_wikipedia.name)
print(search_wikipedia.description)
pprint.pprint(search_wikipedia.args)
print(search_wikipedia("Large language models"))

search_wikipedia
Run Wikipedia search and get page summaries.
{'query': {'title': 'Query', 'type': 'string'}}
Page: Large language model
Summary: A large language model (LLM) is a type of computational model designed for natural language processing tasks such as language generation. As language models, LLMs acquire these abilities by learning statistical relationships from vast amounts of text during a self-supervised and semi-supervised training process.
The largest and most capable LLMs are artificial neural networks built with a decoder-only transformer-based architecture, enabling efficient processing and generation of large-scale text data. Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.

Page: Language model
Summary: A language model is a prob

### Load LLM

In [23]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "/opt/llm/models/Breeze-7B-32k-Instruct-v1_0/"
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=2000,
    do_samle=True,
    temperature=0.1,
)
llm = HuggingFacePipeline(pipeline=pipe)

ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.

### Route and Tool using